In [5]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import sqlite3

In [20]:
# Create database
conn = sqlite3.connect("cairo_weather.db")

conn.execute("CREATE TABLE weather (id INTEGER PRIMARY KEY AUTOINCREMENT, temp int, wind text, humidity int, visibility int, pressure int, time_now datetime)")

conn.close()

OperationalError: table weather already exists

In [25]:
conn = sqlite3.connect("cairo_weather.db")

for i in range(10):
    response = requests.request('GET', 'https://eg.freemeteo.com/weather/cairo/current-weather/location/?gid=360630&language=english&country=egypt')
    soup = BeautifulSoup(response.text, 'html.parser')
    
    div = soup.find('div', attrs={'class':'last-renew-info'})
    temp = int(div.find('div', class_='temp').text.replace('°C', '')) # 1: Temperature
    
    wind = div.find('div', class_='wind')
    white = wind.find('span', class_='white')
    try:
        wind_speed = int(white.find('b').text)  # 2: Wind speed
    except AttributeError:
        wind_speed = white.get_text().split(": ")[1]
    
    stats = div.find(class_='stats')
    stats_list = stats.text.split('|')
    
    humidity = stats_list[0].strip().split('\r\n')[1].strip().replace('%', '')    # 3: Humidity
    visibility = stats_list[1].strip().split('\r\n')[1].strip().replace('m', '')  # 4: Visibility
    pressure = stats_list[2].strip().split('\r\n')[1].strip().split('mb')[0]    # 5: Pressure
    
    date = datetime.now()  # 6: time_now
    
    ###### YOUR CODE HERE
    
    
    conn.execute("INSERT INTO weather (temp, wind, humidity, visibility, pressure, time_now) VALUES (?, ?, ?,?, ?, ?)", [temp, wind_speed, humidity, visibility, pressure, date])
    conn.commit()  
    
    time.sleep(2)
    
conn.close()

In [22]:
#code to show database
import pandas as pd

conn = sqlite3.connect("cairo_weather.db") 
df = pd.read_sql("SELECT * FROM weather", conn) 
conn.close()


df.head()

,id,temp,wind,humidity,visibility,pressure,time_now
0,1,9,Calm,76,6000,1023,2021-12-26 07:53:18.874793
1,2,9,Calm,76,6000,1023,2021-12-26 07:53:22.032150
2,3,9,Calm,76,6000,1023,2021-12-26 07:53:25.450449
3,4,9,Calm,76,6000,1023,2021-12-26 07:53:29.232431
4,5,9,Calm,76,6000,1023,2021-12-26 07:53:32.284418
